# Data Visualization Project 

## Jeremia Neuhaus, Dominic von Olnhausen, Niklas Bräuninger, Janis Hahn

### Einleitung

Auf den folgenden Seiten werden wir die Daten des Datensatzes [Aktuell_Deutschland_SarsCov2_Infektionen.csv](https://github.com/robert-koch-institut/SARS-CoV-2-Infektionen_in_Deutschland), publiziert vom Robert Koch Institut, auswerten. Der Datensatz wird jeden Tag um die neusten Fälle erweitert und zählt stand heute (07.05.2023) 7160131 Zeilen und 12 Spalten. Folgende Spalten sind im Datensatz enthalten: 
- IdLandkreis: Einzige Geoinformation um Fälle dem Landkreis zuzuordnen
- Altersgruppe: Unterteilt die Fälle in Altersgruppen: A00-A04, A05-A14, A15-A34, A35-A59, A60-A79, A80+, unbekannt
- Geschlecht: Männlich und weiblich
- Meldedatum: Datum, an dem der Coronafall gemeldet wurde
- Refdatum: Datum, des Erkrankungsbeginns. Wenn nicht bekannt -> Meldedatum
- IstErkrankungsbeginn: 1: Refatum = Erkrankungsbeginn, 0: Refdatum = Meldedatum  
- NeuerFall
- NeuerTodesfall: -9: Keine Informationen über Krankheitsverlauf vorhanden
- NeuGenesen: -9: Keine Informationen über Krankheitsverlauf vorhanden
- AnzahlFall: Anzahl der gemeldeten Neuinfektionen
- AnzahlTodesfall: Anzahl der gemeldeten Todesfälle 
- AnzahlGenesen: Anzahl der gemeldeten Genesungsfälle

Für unsere Datenvisualisierung verwenden wir aus diesem Datensatz die Spalten: IdLandkreis, Altersgruppe, Meldedatum, Refdatum, AnzahlFall, AnzahlTodesfall und AnzahlGenesen.

Verwendete Files:
- [Aktuell_Deutschland_SarsCov2_Infektionen](https://github.com/robert-koch-institut/SARS-CoV-2-Infektionen_in_Deutschland)
- [Shapefile](https://github.com/Brouny09/DataScience_WI22A2/blob/main/gadm41_DEU_2.shp)
- [Google Trends Daten](https://raw.githubusercontent.com/Brouny09/DataScience_WI22A2/main/Corona_Trends_neu%20(1).csv)
- [Impfdaten](https://raw.githubusercontent.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/master/Aktuell_Deutschland_Bundeslaender_COVID-19-Impfungen.csv)

## Datenvisualisierung

### Datenvorbereitung

Zunächst werden alle benötigten Libraries installiert und anschließend importiert: 

In [56]:
import seaborn as sns
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.dates as mdates
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.colors import LogNorm
from matplotlib import ticker
import numpy as np
import plotly.express as px
from termcolor import colored
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import locale
import warnings
from scipy.signal import savgol_filter

### Corona-Daten des RKI laden
Im nächsten Schritt wurde der Hauptdatensatz des Robert-Koch-Instituts geladen und entsprechend vorbereitet. In der [Dokumentation](https://github.com/robert-koch-institut/SARS-CoV-2-Infektionen_in_Deutschland) unseres Datensatzes ist zu lesen, dass die 12 Stadtbezirke von Berlin als eigene "Landkreise" aufgegliedert wurden . Hier wird von den Vorgaben des AGS abgewichen. Für die Visualisierung unserer Daten, interessieren uns allerdings die Fallzahlen auf Landkreisebene, weshalb wir Berlin als einen Landkreis visualisieren und die Fälle der einzelnen Stadtteile entsprechend aufsummieren. Hierfür ersetzen wir in unserem Datensatz die IdLandkreis der betroffenen Stadtkreise durch die AGS definierte ID 11000.

In [57]:
cov_data = pd.DataFrame()
cod_data = pd.read_csv("https://media.githubusercontent.com/media/robert-koch-institut/SARS-CoV-2-Infektionen_in_Deutschland/main/Aktuell_Deutschland_SarsCov2_Infektionen.csv")
cov_data['Meldedatum'] = pd.to_datetime(cov_data['Meldedatum'], format='%Y-%m-%d')
cov_data['Refdatum'] = pd.to_datetime(cov_data['Refdatum'], format='%Y-%m-%d')
cov_data['IdLandkreis'] = cov_data['IdLandkreis'].astype(int)
cov_data["IdLandkreis"].replace([11001, 11002, 11003, 11004, 11005, 11006, 11007, 11008, 11009, 11010, 11011, 11012], 11000, inplace=True)
#Einlesen der Daten 

KeyError: 'Meldedatum'

In [ ]:
cov_data = pd.DataFrame()
cov_data  = pd.read_csv("/Users/jeremianeuhaus/Desktop/DataScienceProject/Aktuell_Deutschland_SarsCov2_Infektionen.csv")
cov_data['Meldedatum'] = pd.to_datetime(cov_data['Meldedatum'], format='%Y-%m-%d')
cov_data['Refdatum'] = pd.to_datetime(cov_data['Refdatum'], format='%Y-%m-%d')
cov_data['IdLandkreis'] = cov_data['IdLandkreis'].astype(int)
cov_data["IdLandkreis"].replace([11001, 11002, 11003, 11004, 11005, 11006, 11007, 11008, 11009, 11010, 11011, 11012], 11000, inplace=True)
#Einlesen der Daten 

### Geografische Daten laden 

Die geografischen Daten werden in From einer Shapefile dargestellt, die Geodaten basierend auf den Landkreisen von Deutschland enthält. Die Längen und Breitengrade liegen im Format von Multipolygon vor, geben also alle Punkt der Grenzen an. Im Laufe der Arbeit stellte sich heraus, dass die Shapefile etwas veraltet ist und teilweise falsche Informationen über die Landkreise in Deutschland enthält. Das äußerte sich darin, dass für den Landkreis Göttingen und Osterode am Harz keine Daten angezeigt wurden. Der Fehler bestand darin, dass Osterode am Harz am 1. November 2016 mit Göttingen zum neuen Landkreis Göttingen fusionierte. Das war in den Coronadaten richtig hinterlegt, in der Shapefile allderings nicht. Ähnlich verhält es sich mit dem Landkreis Eisenach der seit dem 1. Juni 2021 zum Wartburgkreis gehört. Die LandkreisIds dieser Landkreise mussten in der Shapefile entsprechend manuell angepasst werden. 

In [ ]:
#Geodaten laden und vorbereiten
geoframe = gpd.read_file("gadm41_DEU_shp/gadm41_DEU_2.shp", na_values=['NA', 'NaN', 'N/A'])
#Format anpassen, führende 0 löschen
geoframe['CC_2'] = geoframe['CC_2'].str.lstrip('0')
#NA Werte liegen als String vor. Diese werden herausgefiltert
geoframe = geoframe[geoframe['CC_2'] != 'NA']
#Löschung aller weiteren Na Werte
geoframe = geoframe.dropna(subset=["CC_2"])
#Umbenennung der Spalte "CC_2" in "IdLandkreis" analog des DataFrames dflarge
geoframe = geoframe.rename(columns={"CC_2": "IdLandkreis"})
#Bevölkerungsdaten laden
geoframe['IdLandkreis'] = geoframe['IdLandkreis'].astype(int)
#Göttingen und Osterode am Harz fusionierten am 1. November 2016 zum neuen Landkreis Göttingen, Shapefile Daten veraltet
geoframe.loc[geoframe['NAME_2'] == 'Göttingen', 'IdLandkreis'] = 3159
geoframe.loc[geoframe['NAME_2'] == 'Osterode am Harz', 'IdLandkreis'] = 3159
#Die Stadt Eisenach gehört seit 1. Juli 2021 zum Wartburgkreis. Shapefile enthält entsprechend veraltete Daten.
geoframe.loc[geoframe['NAME_2'] == 'Eisenach', 'IdLandkreis'] = 16063

### Bevölkerungsdaten laden

Für die Kartenplots benötigen wir teilweise die Populationsdaten der Landkreise. Hierfür werden diese im folgenden geladen und als Pandas DataFrame gespeichert. 

In [ ]:
#Bevölkerungsdaten laden und vorbereiten
population_lk= pd.DataFrame()
population_lk = pd.read_excel("/Users/jeremianeuhaus/Downloads/04-kreise.xlsx")
population_lk = population_lk.dropna()
population_lk["IdLandkreis"] = population_lk["IdLandkreis"].astype(int)
# Ändern des Datentyps der Spalte "Bevölkerung" in Integer, um einheitlichen Datentyp für merge zu haben 
population_lk["Bevölkerung"] = population_lk["Bevölkerung"].astype(int)

### Google-Trends Daten laden

Für die dritte Visualisierung benötigen wir die Google Trends Suchanfragen für das Wort "Corona".

In [ ]:
df_Corona_Trends = pd.DataFrame()
df_Corona_Trends  = pd.read_csv("/Users/jeremianeuhaus/Downloads/Corona_Trends_neu (1).csv")

### Impdaten Laden
Für die letzte Visualisierung benötigen wir die Impfdaten. Diese sind ebenfalls im GitHub-Repository des RKIs zu finden.

In [ ]:
df_impf = pd.read_csv("/Users/jeremianeuhaus/Downloads/COVIDImpfungen.csv", sep=",", decimal=".", encoding="latin1", on_bad_lines='skip')
url = "https://raw.githubusercontent.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/master/Aktuell_Deutschland_Bundeslaender_COVID-19-Impfungen.csv"

## 1. Visualisierung
### Kartenvisualisierung basierend auf den Fallzahlen
Im folgenden werden die Daten aus Landkreis_Bev und cov_data zusammengefügt. Anschließend wird basierend auf den beiden Columns "AnzahlFall" und "Bevölkerung" der Wert "Percentage" berechnet der die Anzahl der Fälle in das Verhältnis mit der gesamten Population des Landkreises setzt.  

In [ ]:
#merge der DataFrames cov_data und Landkreis_Bev basierend auf den Werten der Spalte "IdLandkreis"
merged_cov_data = pd.merge(cov_data , population_lk, on='IdLandkreis')
#Neuen nach IdLandkreis gruppierten DataFrame erstellen und Anzahlfälle summieren. Entsprechende Bevölkerungszahl hinzufügen
merged = merged_cov_data.groupby('IdLandkreis').agg({'AnzahlFall': 'sum', 'Bevölkerung': 'first'}).reset_index()
#Prozentuale Infektionsrate ausrechnen und im DataFrame unter der Spalte "Percentage" speichern 
merged['Percentage'] = merged['AnzahlFall']/merged['Bevölkerung']*100

# Fälle gruppieren und zählen basierend auf Landkreis
grouped_cases = merged.groupby('IdLandkreis')['AnzahlFall'].sum().reset_index()

# Neue Spalte erstellen mit den logarithmischen Werten von AnzahlFall, um die mittleren Werte besser darzustellen
merged_map = geoframe.merge(grouped_cases, on='IdLandkreis', how='left')

Für die Visualisierung benutzen wir die selbe Farbskala, die sich aus den Farben rot, orange und grün zusammengesetzt. Die Skala ist sehr intuitiv zu lesen: Rot steht für hohe Fallwerte und grün für wenige Fallwerte.

In [ ]:
colorscale = colors.LinearSegmentedColormap.from_list('colorscale', ['green', 'orange', 'red'])

Anschließend wird der eigentliche Plot erstellt. Wir haben uns in diesem Fall für eine logarithmische Skalierung entschieden, da es viele Landkreise gibt, die ähnliche Werte haben mit Ausnahme einiger Aureißwerte wie z.B. Berlin.

In [ ]:
merged_map['per_hundret_k'] = round((merged_map['AnzahlFall']/83000000) *100000).astype(int)
norm = colors.LogNorm(vmin=merged_map['per_hundret_k'].min(), vmax=merged_map['per_hundret_k'].max())
# Plotten der Deutschlandkarte mit den aggregierten Fallzahlen
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Deaktiviere den UserWarning für das centroid-Problem
warnings.filterwarnings("ignore", category=UserWarning)

merged_map.plot(ax=ax, column='per_hundret_k', cmap=colorscale, linewidth=0.8, edgecolor='0.8', vmin=0, vmax=merged_map['per_hundret_k'].max(), legend=False, norm=norm)


top_five = merged_map.nlargest(5, "per_hundret_k")[["NAME_2", "per_hundret_k", "geometry"]]
top_five['Coords'] = top_five['geometry'].centroid
top_five['Coords'] = top_five['Coords'].apply(lambda p: (p.x, p.y))

# Pfeile und Beschriftungen für die Top 5 Werte erstellen
for row in top_five.itertuples():
    lon, lat = row.Coords
    text = row.NAME_2
    per_hundret_k = row.per_hundret_k
    ax.annotate("", xy=(lon, lat), xytext=(lon+0.3, lat+0.3), arrowprops=dict(arrowstyle="-"))
    ax.text(lon+0.3, lat+0.3, f'{text}\n({per_hundret_k} Fälle)', transform=ccrs.PlateCarree())

plt.title('Absolute Fallzahlen pro Landkreis in Fällen pro 100.000 Einwohnern',fontsize=15, pad=20, fontweight='bold', loc='left')

# Manuell festgelegte Ticks in der Farblegende
cbar = plt.colorbar(ax.get_children()[0], ax=ax, orientation='vertical', aspect=50)
cbar.set_label('Anzahl der Fälle pro 100.000 Einwohner')

ticks = [round(merged_map['per_hundret_k'].min()),25,50, 75, 100,150, 200, 250, 300, 400, 500, 600, 800, 1000,1500, round(merged_map['per_hundret_k'].max()) ]
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks)

plt.show()




### Interpretation
Für die bessere Übersicht wurden noch die 5 Landkreise mit den meisten Coronafällen markiert. Auch ein Blick auf die Karte zeigt schnell dass die Landkreise Berlin, Hamburg, München, Hannover und Köln rot eingefärbt sind. Außerdem erkennt man dass die Landkreise die an der Grenze liegen eher selten weniger Coronafälle haben. So gut wie alle Landkreise an der Grenze sind mindestens grün-gelb gefärbt. Das ergibt auch Sinn, da an den Grenzen vermutlich am meisten Verkehr ist und viele Menschen, die nach Deutschland einreisen über diese Landkreise einreisen. Was auch auffällt ist, dass eher die geografisch kleineren Landkreise grün eingefärbt sind. Man könnte das darauf zurückführen, dass weniger Fläche auch weniger Menschen bedeutet, und somit auch weniger potentielle Ansteckungsmöglichkeiten. Dieser Ansatz wird im nächsten Kartenplot nochmal genauer untersucht. 

## 2. Visualisierung
### Kartenvisualisierung basierend auf den Fallzahlen im Verhätlnis zur Bevölkerung 
Die Erstellung der Virtualisierung ist größtenteils analog der ersten Kartenvisualisierung. Der Unterschied besteht darin, dass die Karte basierend auf den prozentualen Werten in der Spalte "Percentage" eingefärbt wird. Dabei werden die Anzahl der Fälle pro Landkreis ins Verhältnis zur Bevölkerung des Landkreises gesetzt, um eine andere Perspektive auf die Fallzahlen zu bekommen.
Auch hier haben wir uns für eine logarithmische Skalierung entschieden, da hier zwar andere Werte visualisiert werden, die Verteilung dieser Werte jedoch gleich bleibt.

In [ ]:
#Vorbereiteten DataFrame "merged" mit dem Geopandas Dataframe "geoframe" mergen, um geodaten zu ergänzen
merged_map = geoframe.merge(merged, on='IdLandkreis', how='left')     
merged_map['Percentage'] = round(merged_map['Percentage'], 1)
      
#Skala an den Min und Max werten der Spalte "Percentage ausrichten, norm auf LogNorm setzen, da Werte im Unteren bereich alle ähnlich"
norm = colors.LogNorm(vmin=merged_map['Percentage'].min(), vmax=merged_map['Percentage'].max())

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    
top_five = merged_map.nlargest(5, "Percentage")[["NAME_2", "Percentage", "geometry"]]
top_five['Coords'] = top_five['geometry'].centroid
top_five['Coords'] = top_five['Coords'].apply(lambda p: (p.x, p.y))
merged_map.plot(ax=ax, column='Percentage', cmap=colorscale, linewidth=0.8, edgecolor='0.8', vmin=0, vmax=merged_map['Percentage'].max(), legend=False, norm=norm)
    
#Beschriftung so skalieren dass es erkenntlich ist. 
i = 0.2
for row in top_five.itertuples():
    lon, lat = row.Coords
    text = row.NAME_2
    percentage = row.Percentage
    ax.annotate("", xy=(lon, lat), xytext=(lon+0.3, lat+0.3+i), arrowprops=dict(arrowstyle="-"))
    ax.text(lon+0.3, lat+0.3+i, f'{text}\n({percentage}%)', transform=ccrs.PlateCarree())
    i+=0.2
        
cbar = plt.colorbar(ax.get_children()[0], ax=ax, orientation='vertical', aspect=50)
cbar.set_label('Anzahl der Fälle in Prozent')

ticks = [round(merged_map['Percentage'].min()), 40, 50, 60]  # Manuell festgelegte Ticks

cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks)

plt.title('Fallzahlen pro Landkreis (eingefärbt entsprechend den Werten)')

plt.show()

### Interpretation
Man sieht schnell, dass dieser Plot eine ganz andere Realität darstellt. Die Landkreise mit den fünf höchsten Infektionsraten sind Vechta, Fulda, Mühldorf am Inn, Emsland und Cloppenburg. Alles Landkreise die in der vorherigen Visualisierung keine auffallend hohen Fallzahlen aufwiesen. Die top fünf Landkreise Berlin, Hamburg, München, Hannover und Köln aus der ersten Visualisierung ordnen sich in den mittleren Prozentbereichen ein. 
Was darüber hinaus auffällt ist, dass das Bundesland Bayern die höchsten prozentualen Infektionsraten hat. 

## 3. Visualisierung 
### Liniendiagramm der Fallzahlen inkl. Google Trends und Lockdown
Neben der Kartenvisualisierung auf Bundselandebene und Landkreisebene, möchten wir den gesamten verlauf der COVID-19 Fälle über die Jahre 2020 bis 2023 veranschaulichen. Um ein Veränderung über die Zeit kenntlich zu machen, erwies sich die Viusalisierung mittels eines Liniendiagrammes am besten.
Wichtig zu Berücksichtigen ist, das die Anzahl der Fälle pro Tag in dem unten dargestellten Liniendiagramm vorhanden sind. Die Anzahl der Fälle auf der y-Achse bezieht sich auf pro 100.000 Menschen. Auf der zweiten y-Achse visualisieren wir die Corona-Trend-Suchanfragen, sprich wie oft wurde zu gegebenem Datum der Begriff Corona gesucht. Diese Werte liegen lediglich in prozentualen Anteilen vor, da die genaue Anzahl der Suchen nicht herausgegeben wird.
Des Weiteren haben wir die Corona-Lockdown Zeiträume mit zwei unterschiedlich markierten rechtecken kenntlich gemacht. Da der Zeitraum relativ breit ist, sind diese nicht immens groß aber man kann dennoch deutlich den Einfluss der Lockdowns auf die Corona-Fallzahlen erkennen.

In [ ]:
import warnings
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

warnings.filterwarnings("ignore", category=UserWarning)

colors = sns.color_palette("bright")
colorBlau = colors[0]
colorGrau = colors[7]
colorGreen = colors[2]
colorRed = colors[3]

AnzahlFall_Meldedatum = cov_data.groupby('Meldedatum')['AnzahlFall'].sum() / 83000000 * 100000

fig, ax = plt.subplots(figsize=(35, 10))
ax2 = ax.twinx()
ax2.set_ylim(0, 100)
ax2.set_ylabel('Google Trends Suchanfragen', fontsize=20)

sns.lineplot(x=AnzahlFall_Meldedatum.index, y=AnzahlFall_Meldedatum.values, ax=ax, color=colorGrau, alpha=0.3)
plt.plot(df_Corona_Trends['Woche'], df_Corona_Trends['Corona'], color=colorBlau)
AnzahlFall_Meldedatum.rolling(7).mean().plot(ax=ax, color='black')

plt.title('Anzahl der COVID-19 Fälle verglichen mit Google Trends Anfragen', fontsize=25, pad=20, fontweight='bold', loc='left')

ax.set_xlabel('Meldedatum', fontsize=20)
ax.set_ylabel('Anzahl der Fälle pro 100.000', fontsize=20)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%m-%Y"))

ax.xaxis.grid(False)
ax.yaxis.grid(False)

lockdown_start1 = pd.to_datetime('2020-03-22')
lockdown_end1 = pd.to_datetime('2020-05-04')
lockdown_start2 = pd.to_datetime('2020-12-13')
lockdown_end2 = pd.to_datetime('2021-01-10')
lockerungen_start = pd.to_datetime('2021-03-04')
lockerungen_end = pd.to_datetime('2021-03-05')

ax.axvspan(lockdown_start1, lockdown_end1, alpha=0.3, color=colorRed)
ax.axvspan(lockdown_start2, lockdown_end2, alpha=0.3, color=colorRed)
ax.axvspan(lockerungen_start, lockerungen_end, alpha=0.3, color=colorGreen)

start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2023-06-01')
ax.set_xlim(start_date, end_date)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

ax.set_ylim(0, max(AnzahlFall_Meldedatum.max(), df_Corona_Trends['Corona'].max()) * 1.1)

plt.show()


### Interpretation
Wie man sehen kann, ist der verlauf der Corona Fallzahlen pro Tag zu beginn von Corona noch sehr gering. Deutschland hat relativ schnell reagiert und mittels zweier Lockdowns die Infizierungen drastisch gemindert.
Die blaue Google Trends Linie zeigt ziemlich gut das Interesse der Gesamtbevölkerung zu dem Thema Corona. Zu Beginn als Corona in unser Leben trat, erreichte die Suchanfragen zwischen dem 01.03.2020 und dem 01.04.2020
die größte Anzahl an Suchanfragen. Über die Zeit kann man wahrnehmen, das die Google Trendslinie immer dann ansteigt, wenn auch die Anzahl der Fallmeldungen zunimmt. Gegen Ende als auch die Lockerungen und Impfungen eintraten, 
wurde das Thema wieder vermehrt unrelevant. 
Ziemlich signifikant ist auch der extreme Antieg der COVID-19 Meldungen nach den Lockerungen extrem in die Höhe geschossen. Dies is dem Fakt geschuldet, das die Menschen sich wieder trafe, Kontakte Pflegten, feiern gingen und sich 
hierdurch vermehrt ansteckten.

## 4. Visualisierung 
### Balkendiagramm Corona Impfungen 

Insgesamt erweist sich die Entscheidung darüber, wer zuerst geimpft werden sollte, als eine sehr komplexe Angelegenheit, die wissenschaftliche, politische, ethische und logistische Faktoren berücksichtigen muss. Es erfordert eine enge Zusammenarbeit von Regierung, Gesundheitsbehörden und der Öffentlichkeit, um sicherzustellen, dass die Impfstrategien effektiv und fair sind.

Um den gesamten Prozess zu vereinfachen haben wir uns dazu entschieden eine Visualisierung in Form eines Balkendiagramms vorzunehmen, welche die COVID-19 Fälle pro Altersgruppe anzeigt. Um die Verteilung der Todesfälle in den Altersgruppen darzustellen, wurde daneben ein weiteres Balkendiagramm dargestellt.
Diese Lösung kam zum Einsatz, da die Anzahl der Todesfälle zu gering ist, um neben den Gesamtfällen innerhalb eine Balkendiagrammes zu stehen. Durch die Skalierung waren die Todesfälle kaum ersichtlich.

In [ ]:
# Wir h aus df_Alter_Faelle und df_Alter_TodesFaelle
x = df_Alter_Faelle[df_Alter_Faelle['Altersgruppe'] != 'unbekannt']['Altersgruppe']
y1 = df_Alter_Faelle[df_Alter_Faelle['Altersgruppe'] != 'unbekannt']['AnzahlFall'] / 83000000 * 100000
y2 = df_Alter_TodesFaelle[df_Alter_TodesFaelle['Altersgruppe'] != 'unbekannt']['AnzahlTodesfall']

# Farbpalette "bright" verwenden und Farben für die Plots festlegen
colors = sns.color_palette("bright")
colorBlau = colors[0]  # Blau
colorRed = colors[3]  # Rot
colorGray = colors[7] #Grey

# Setze locale für Tausender-Trennpunkt
locale.setlocale(locale.LC_ALL, '')

# Erstelle die Figur und Achsen-Objekte
fig, ax = plt.subplots(1, 2, figsize=(20, 8))

# Erster Barplot auf der linken Achse
sns.barplot(x=x, y=y1, ax=ax[0], color=colorBlau)
ax[0].set_title('Anzahl der COVID-19 Fälle pro 100,000', fontsize=20, fontweight='bold', pad=20, loc='left')
ax[0].set_xlabel('Altersgruppe', fontsize=15)
ax[0].set_ylabel('Anzahl Fälle pro 100.000 Einwohner', fontsize=15)

# Text für die Anzahl der Fälle über den Bars im ersten Plot
for i, yval in enumerate(y1):
    formatted_val = locale.format_string("%d", yval, grouping=True)  # Formatieren der Anzahl mit Tausender-Trennpunkt
    ax[0].text(i, yval, formatted_val, ha='center', va='bottom')

# Skalierung der y-Achse auf ganze Zahlen
ax[0].yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

# Zweiter Barplot auf der rechten Achse
sns.barplot(x=x, y=y2, ax=ax[1], color=colorRed)
ax[1].set_title('Anzahl der Todesfälle pro Altersgruppe', fontsize=20, fontweight='bold', pad=20, loc='left')
ax[1].set_xlabel('Altersgruppe', fontsize=15)
ax[1].set_ylabel('Anzahl Todesfälle', fontsize=15)

# Text für die Anzahl der Fälle über den Bars im zweiten Plot
for i, yval in enumerate(y2):
    formatted_val = locale.format_string("%d", yval, grouping=True)  # Formatieren der Anzahl mit Tausender-Trennpunkt
   
ax[1].text(i, yval, formatted_val, ha='center', va='bottom')

# Abstände der x und y-Achsen Labels zur achse selbst
ax[0].xaxis.set_label_coords(0.5, -0.1)#
ax[1].xaxis.set_label_coords(0.5, -0.1)
ax[0].yaxis.set_label_coords(-0.1, 0.5)
ax[1].yaxis.set_label_coords(-0.1, 0.5)  

# Gitter entfernen
ax[0].grid(False)
ax[1].grid(False)



# Zeige den Plot
plt.show()

### Interpretation
Aufgrund der zugrunde liegenden Visualisierung ist ziemlich signifikant, das die Altersgruppe A35-A59 die meisten und die Altergruppe A00-A004 die wenigsten Fälle aufweist. Verglichen mit dem rechten Balkendiagramm sieht man deutlich, das sich die Todesfälle dieser Altergruppen ziemlich im Rahmen halten und nicht herausstechen. 
Interessanter ist, das anhand der Todesfälle ziemlich erkenntlich ist, das die Sterberate in Richtung der älteren Generationen deutlich höher ist, als die der jüngeren Generationen.
Aufgrund dieser Information erkennt Sie, das die alten Menschen ein deutlich höheres Todesrisiko aufweisen, als jüngere Generationen. 
Basierend auf dieser Tatsache, wäre es sinnvoll zuerst die älteren Bevölkerungsschichten zu isolieren und ihnen so schnell wie möglich die erste Impfung zu verabreichen, um diese zu schützen. 
Anschließen wäre es wichtig die altersgruppen A15-A34 und A35-A59 zu impfen, da diese im altäglichen leben deutlich mehr unternehmen und dadurch den Virus schneller verbreiten.


## 5. Visualisierung
### Verwendung der Coronaimpfstoffe
Zum Abschluss werden die Daten aus dem Impfdatensatz des Robert-Koch-Instituts dargestellt. Dabei wird in der ersten Visualisierung eine zeitliche Einordnung der Impfserien gegeben. In der zweiten Visualisierung wird die jeweilige Häufigkeitsverteilung der verwendeten Impfstoffe dargestellt.

In [ ]:
plt.figure(figsize=(14, 14))

plt.subplot(2, 1, 1)
df_impf['Impfdatum'] = pd.to_datetime(df_impf['Impfdatum'])
serien = df_impf.groupby(['Impfdatum', 'Impfserie'])['Anzahl'].sum().unstack()
palette = sns.color_palette("bright", serien.shape[1])
styles = ['-', '-', '-', '-']
for i, serie in enumerate(serien.columns):
    if i < len(styles) and i < len(palette):
        smoothed_data = savgol_filter(serien[serie], window_length=21, polyorder=2)
        plt.plot(serien.index, smoothed_data, label=f'Impfserie {serie}', linestyle=styles[i], color=palette[i])
plt.xlabel('Impfdatum', fontsize=15)
plt.ylabel('Anzahl', fontsize=15)
plt.title('Zeitlicher Verlauf der Impfserien (geglättet)', fontsize=20, pad=20, fontweight='bold', loc='left')
plt.legend(title='Impfserie')
plt.xticks(rotation=45)
plt.yticks(fontsize=10)
plt.ticklabel_format(style='plain', axis='y')

plt.subplot(2, 1, 2)
total_sum = df_impf.groupby('Impfdatum')['Anzahl'].sum()
top_6_impfstoffe = df_impf.groupby('Impfstoff')['Anzahl'].sum().nlargest(6).index
top_6_data = df_impf[df_impf['Impfstoff'].isin(top_6_impfstoffe)]
impfstoff_datum_pct = top_6_data.groupby(['Impfstoff', 'Impfdatum'])['Anzahl'].sum() / total_sum * 100
palette = sns.color_palette("bright", len(top_6_impfstoffe))
for i, impfstoff in enumerate(top_6_impfstoffe):
    impfstoff_data = impfstoff_datum_pct.loc[impfstoff]
    smoothed_data = savgol_filter(impfstoff_data.values, window_length=15, polyorder=2)
    smoothed_data = np.maximum(smoothed_data, 0)
    plt.plot(impfstoff_data.index, smoothed_data, label=impfstoff, color=palette[i])
handles, labels = plt.gca().get_legend_handles_labels()
order = sorted(range(len(labels)), key=lambda k: labels[k])
plt.legend([handles[i] for i in order], [labels[i] for i in order])
plt.xlabel('Impfdatum', fontsize=15)
plt.ylabel('Prozentualer Anteil', fontsize=15)
plt.title(' Trend der Top 6 Impfstoffe (geglättet)', fontsize=20, pad=20, fontweight='bold', loc='left')
plt.xticks(rotation=45)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

### Interpretation
In der oberen Grafik wird die Anzahl der Impfungen in der jeweiligen Impfserie im zeitlichen Verlauf dargestellt. Man sieht hier sehr gut, wie sich die Impfungen zeitlich aus verschiedenen Gründen verteilen. Die kurven, der Impfserie 1 und 2 sind flacher und über einen längeren Zeitraum. Bei der 3. Impfung konnte die hohe Nachfrage schneller.

In der Grafik darunter wird der Marktanteil der Impfstoffarten prozentual dargestellt. Hier kann man gut erkennen, wie Corminaty (Biontech) bis auf einen Spitzenwert von Spikevax (Moderna) im absoluten Höchstwert der Impfungen immer der Marktführer ist. Im späteren zeitlichen Verlauf kann man die Substitutionen der neuen Varianten von Corminaty sehen. Allgemein ist bei einer hoher Impfrate immer ein Einbruch von Corminaty zu sehen, der darauf schließen lässt, dass dieser die hohe Nachfrage nicht allein Abdecken konnte.
